### Notebook to adjust PetroNER for Entity Linking task

In [38]:
from owlready2 import *
import gensim
from conllu import parse_incr, parse
import numpy as np

Load ontology and embeddings

In [39]:
#OntoGeoLógica populada (OntoGeoLogicaPopulada.owl)
onto = get_ontology("../../KnowledgeGraph/OntoGeoLogicaPovoadaInstanciasRelacoes.owl")
onto.load()

get_ontology("http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#")

In [40]:
# Modelo OWL2Vec - 
PetroOntoVec = gensim.models.Word2Vec.load("../../Embeddings/PetroOntoVec/PetroOntoVec_simples/outputontology.embeddings")

Load PetroNER (file in CONLLU format)

In [41]:
# CONLLU parser

# Corpora treino
PetroNER_treino = open("../PetroNER/petroner-Treino-uri-2022-03-07.conllu", "r", encoding="utf-8")
PetroNER_treino_sentences = parse_incr(PetroNER_treino)

# Corpora validação
PetroNER_valid = open("../PetroNER/petroner-Validação-uri-2022-03-07.conllu", "r", encoding="utf-8")
PetroNER_valid_sentences = parse_incr(PetroNER_valid)

# Corpora teste
PetroNER_teste = open("../PetroNER/petroner-Teste-uri-2022-03-07.conllu", "r", encoding="utf-8")
PetroNER_teste_sentences = parse_incr(PetroNER_teste)

### Preparing and processing the dataset

Função que recebe o texto anotado em formato CONLLU e processa para o treinamento. Tem como saída as sentenças acrescida das tags para indicar as entidades, bem como com a entidade no início do texto; lista com as entidades (instâncias extraídas do texto); lista com as classes que as entidades pertencem; e lista com as URI referente às entidades. O texto sai com o seguinte formato:  
"[Nome da classe] Nome da entidade | blablabla blablabla blablabla [E] Nome da entidade [/E] blablabla blablabla." 

In [42]:
# Função que recebe
def preparar_dataset(tokenlist):

    #sent = TokenList([])
    sent_orig = ''   # String para compor a sentença original
    sents_NER = []   # Lista para armazenar as sentenças com indicação das entidades
    NER_on = False   # Variável para indicar se está iterando por token de entidades
    NER_tok = ''     # string para armazenar os tokens das entidades que estão sendo iteradas
    MW_on = False    # Variável para indicar se está iterando por tolkens multiword
    MW_tok = ''      # string para armazenar o termo multiword
    MW_end = 0       # ID final do token multiword
    ENT = []         # Lista para armazenar as entidades encontradas no texto
    ENT_class = []   # Lista para armazenar as classes das entidades encontradas no texto
    ENT_URI = []     # Lista para armazenar as URI das entidades encontradas no texto

    #Iterando por todos os tokens de uma sentença
    for tok in tokenlist:
        # ignorando id não inteiro (tuplas), ou seja token multiword
        if type(tok['id']) == int:

            # Verifica se o token inicia uma entidade ("B")
            if tok['deps'][0] == 'B':
                
                # Verifica se a variável NER_on está ativado (No caso de um tok "B" seguido de outro)
                if NER_on == True:
                    
                    NER_on = False  # Desativa a variável NER_on

                    # Para sentença mais recente da lista sents_NER, inclui a entidade no início e o marcador [/E] no final 
                    sents_NER[-1] = '[' + ENT_class[-1] + '] ' + NER_tok + '| ' + sents_NER[-1].strip() + ' [/E]'

                    # Incluir o NER_tok na lista ENT
                    ENT.append(NER_tok.strip())

                    NER_tok = '' # Apaga a string NER_tok

                NER_on = True  # Ativa a variável NER_on
                # Acrescenta uma sentença com o marcador [E] na listas de sentenças com NER
                sents_NER.append(sent_orig + ' [E]')
                # Incluir classe na lista ENT_class
                ENT_class.append(tok['deps'][2:])
                # Incluir URI da entidade na lista ENT_URI. Caso não haja URI, incluir "None". 
                try:
                    ENT_URI.append(tok['misc']['grafo'])
                except:
                    ENT_URI.append(None)

            # Identifica o primeiro token "O" após uma sequência de entidades
            if (tok['deps'][0] == 'O') & NER_on:
                
                NER_on = False  # Desativa a variável NER_on

                # Para sentença mais recente da lista sents_NER, 
                # inclui a classe e a entidade no início e o marcador [/E] no final 
                sents_NER[-1] = '[' + ENT_class[-1] + '] ' + NER_tok + '| ' + sents_NER[-1].strip() + ' [/E]'

                # Incluir o NER_tok na lista ENT
                ENT.append(NER_tok.strip())

                NER_tok = '' # Apaga a string NER_tok
                
            # Identifica se o último token da sequência tem o marcador "B" ou "I".    
            if (tokenlist[-1]['id'] == tok['id']) & ((tok['deps'][0] == 'B') | (tok['deps'][0] == 'I')):
                NER_on = False  # Desativa a variável NER_on

                # Para sentença mais recente da lista sents_NER, 
                # inclui a classe e a entidade no início e o marcador [/E] no final 
                sents_NER[-1] = '[' + ENT_class[-1] + '] ' + NER_tok + '| ' + sents_NER[-1].strip() + ' [/E]'

                # Incluir o NER_tok na lista ENT
                ENT.append(NER_tok.strip())

                NER_tok = '' # Apaga a string NER_tok

            # Verifica se está iterando por uma multiword ou não
            if not MW_on:

                # Acrecenta o token à string da entidade
                if NER_on:
                    NER_tok = NER_tok + tok['form'] + ' '

                # Se o token for uma pontuação, não inclui espaço entre o token anterior 
                if tok['upos'] == 'PUNCT':
                    # O token é incluído na senteça original e em todas as sentenças da lista sents_NER
                    sent_orig = sent_orig + tok['form']
                    for i in range(len(sents_NER)):
                        sents_NER[i] = sents_NER[i] + tok['form']
                else:
                    sent_orig = sent_orig + ' ' + tok['form']
                    for i in range(len(sents_NER)):
                        sents_NER[i] = sents_NER[i] + ' ' + tok['form']

            else:
                # Se o token for o último termo da multiword, desativamos a variável MW_on
                if tok['id'] == MW_end:
                    MW_on = False

                    # Acrecenta a multiword à string da entidade
                    if NER_on:
                        NER_tok = NER_tok + MW_tok + ' '

                    #Ao invés de incluir nas senteças o token, vamos incluir o MW_tok 
                    # Se o token for uma pontuação, não inclui espaço entre o token anterior 
                    if tok['upos'] == 'PUNCT':
                        # O token é incluído na senteça original e em todas as sentenças da lista sents_NER
                        sent_orig = sent_orig + MW_tok
                        for i in range(len(sents_NER)):
                            sents_NER[i] = sents_NER[i] + MW_tok
                    else:
                        sent_orig = sent_orig + ' ' + MW_tok
                        for i in range(len(sents_NER)):
                            sents_NER[i] = sents_NER[i] + ' ' + MW_tok

        # Se a ID do token for referente a uma multiword, ativamos a variável MW_on, 
        # armazenamos o termo MW_tok e o valor da 'id' final
        else:
            MW_on = True
            MW_tok = tok['form']
            MW_end = tok['id'][-1]
    
    return(sents_NER, ENT, ENT_class, ENT_URI)

In [43]:
def ent_class_URI(sentences):

    # Separando as listas de sentenças, entidades, classes e URI
    text = []
    entities = []
    classes_NER = [] 
    ENT_URI = []
    entities_sem_URI = []

    # Iterando por todas as sentenças em formato CONLLU para preparar o dataset
    n = 0
    m = 0
    
    for tokenlist in sentences:
        # contador de sentenças
        n = n + 1
        # processando cada sentenças
        text_n, entities_n, classes_NER_n, ENT_URI_n = preparar_dataset(tokenlist)
        # Contador de sentenãs com entidades
        if len(text_n) > 0:
            m = m + 1
        
        text = text + text_n
        entities = entities + entities_n
        classes_NER = classes_NER + classes_NER_n
        ENT_URI = ENT_URI + ENT_URI_n

    print('Total de sentenças no arquivo: ', n)
    print('N° de sentenças com entidades: ', m, 'que equivale a ', (m/n)*100, '% do total')
    print ('N° de entidades: ', len(entities))
    print('N° de classes: ', len(classes_NER))
    print('Média de ', len(entities)/m, 'entidades por sentença (das sentenças que possuem entidades)')
    
    #Verificando entidades sem URI
    for i in range(len(ENT_URI)):
        if ENT_URI[i] == None:
            entities_sem_URI.append(entities[i])
        
    print('N° de entities com URI: ', len(ENT_URI) - len(entities_sem_URI))
    print('N° de entities sem URI: ', len(entities_sem_URI))
    print((len(entities_sem_URI)/len(ENT_URI))*100, '% das entidades não possuem URI.')
    
    return (text, entities, classes_NER, ENT_URI, entities_sem_URI)

In [ ]:
print('\nPetroNER - Treino')
treino_text, treino_entities, treino_classes_NER, treino_ENT_URI, treino_entities_sem_URI = ent_class_URI(PetroNER_treino_sentences)

print('\nPetroNER - Validação')
valid_text, valid_entities, valid_classes_NER, valid_ENT_URI, valid_entities_sem_URI = ent_class_URI(PetroNER_valid_sentences)

print('\nPetroNER - Teste')
teste_text, teste_entities, teste_classes_NER, teste_ENT_URI, teste_entities_sem_URI = ent_class_URI(PetroNER_teste_sentences)


PetroNER - Treino


### Buscando vetores do PetroOntoVec para cada URI

In [ ]:
def buscando_URI_vec(text, entities, classes_NER, ENT_URI):

    # Separando as listas de sentenças, entidades, classes, URI e vetores
    text_new = []
    entities_new = []
    classes_NER_new = [] 
    ENT_URI_new = []
    URI_Vec_new = []

    URI_ausente_Onto = []

    n = 0 
    #m = 0
    #Iterando por todas as sentenças, desconsiderando aquelas cujas entidades não possuem URI
    for i in range(len(text)):
        n = n + 1
        if ENT_URI[i] != None:
            # Quebrando as entidades que possuem mais de um URI (ex: "#Aptian,#Miocene" e "#Cenozoic,#Mesozoic")
            ENT_URI_n = ENT_URI[i].split(',#')

            # Iterando pelas ENT_URI após serem quebradas e acrescentando '#" naquelas que perderam na operação de split 
            for ent in ENT_URI_n:
                if ent[0] != '#':
                    ent = '#' + ent
                try:
                    #Acrescentar a URI da entidade e o vetor da URI nas listas
                    URI_Vec_new.append(PetroOntoVec['http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2' + ent])
                    ENT_URI_new.append(ent)

                    # O texto, a entidade no texto e a classe são repetidas quando existem multiplas URI para uma mesma entidade
                    text_new.append(text[i])
                    entities_new.append(entities[i])
                    classes_NER_new.append(classes_NER[i])
                except:
                    #print("URI presente no arquivo CONLLU mas ausente na Ontologia: ", ent)
                    URI_ausente_Onto.append(ent)

    # Transformando as listas em Numpy array
    text = np.array(text_new)
    entities = np.array(entities_new)
    classes_NER = np.array(classes_NER_new) 
    ENT_URI = np.array(ENT_URI_new)
    URI_Vec = np.array(URI_Vec_new)
    URI_ausente_Onto = np.array(URI_ausente_Onto)

    print('Total de sentenças anotadas: ', n)
    print('N° de sentenças: ', len(text))
    print ('N° de entidades: ', len(entities))
    print('N° de classes: ', len(classes_NER))
    print('N° de entidades_ID: ', len(ENT_URI))       
    print('N° de vetores de URI: ', len(URI_Vec))       
    
    return(text, entities, classes_NER, ENT_URI, URI_Vec, URI_ausente_Onto)

In [ ]:
#Treino
(treino_text,
 treino_entities,
 treino_classes_NER,
 treino_ENT_URI,
 treino_URI_Vec,
 treino_URI_ausente_Onto) = buscando_URI_vec(treino_text,
                                             treino_entities,
                                             treino_classes_NER,
                                             treino_ENT_URI)

#Treino
(valid_text,
 valid_entities,
 valid_classes_NER,
 valid_ENT_URI,
 valid_URI_Vec,
 valid_URI_ausente_Onto) = buscando_URI_vec(valid_text,
                                            valid_entities,
                                            valid_classes_NER,
                                            valid_ENT_URI)

#Treino
(teste_text,
 teste_entities,
 teste_classes_NER,
 teste_ENT_URI,
 teste_URI_Vec,
 teste_URI_ausente_Onto) = buscando_URI_vec(teste_text,
                                            teste_entities,
                                            teste_classes_NER,
                                            teste_ENT_URI)

In [ ]:
print('URI no PetroNER_Treino ausentes da ontologia: ')
print(set(treino_URI_ausente_Onto))
print('')
print('URI no PetroNER_Validação ausentes da ontologia: ')
print(set(valid_URI_ausente_Onto))
print('')
print('URI no PetroNER_Teste ausentes da ontologia: ')
print(set(teste_URI_ausente_Onto))

### Salvando dataset

In [ ]:
#Treino

with open('treino - sentences.npy', 'wb') as f:
    np.save(f, treino_text)
with open('treino - entities.npy', 'wb') as f:
    np.save(f, treino_entities)
with open('treino - classes.npy', 'wb') as f:
    np.save(f, treino_classes_NER)
with open('treino - URI.npy', 'wb') as f:
    np.save(f, treino_ENT_URI)
with open('treino - URI_vectors.npy', 'wb') as f:
    np.save(f, treino_URI_Vec)
with open('treino - URI_ausente_Ont.npy', 'wb') as f:
    np.save(f, treino_URI_ausente_Onto)
    
#Validação

with open('valid - sentences.npy', 'wb') as f:
    np.save(f, valid_text)
with open('valid - entities.npy', 'wb') as f:
    np.save(f, valid_entities)
with open('valid - classes.npy', 'wb') as f:
    np.save(f, valid_classes_NER)
with open('valid - URI.npy', 'wb') as f:
    np.save(f, valid_ENT_URI)
with open('valid - URI_vectors.npy', 'wb') as f:
    np.save(f, valid_URI_Vec)
with open('valid - URI_ausente_Ont.npy', 'wb') as f:
    np.save(f, valid_URI_ausente_Onto)
    
#Teste

with open('teste - sentences.npy', 'wb') as f:
    np.save(f, teste_text)
with open('teste - entities.npy', 'wb') as f:
    np.save(f, teste_entities)
with open('teste - classes.npy', 'wb') as f:
    np.save(f, teste_classes_NER)
with open('teste - URI.npy', 'wb') as f:
    np.save(f, teste_ENT_URI)
with open('teste - URI_vectors.npy', 'wb') as f:
    np.save(f, teste_URI_Vec)
with open('teste - URI_ausente_Ont.npy', 'wb') as f:
    np.save(f, teste_URI_ausente_Onto)